In [147]:
import pandas as pd
import numpy as np


In [149]:
df =pd.read_csv('tested.csv')

In [151]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [153]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [155]:
df.shape

(418, 12)

In [157]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [159]:
df.duplicated().sum()

0

In [161]:
df['Cabin'].value_counts()

Cabin
B57 B59 B63 B66    3
B45                2
C89                2
C55 C57            2
A34                2
                  ..
E52                1
D30                1
E31                1
C62 C64            1
C105               1
Name: count, Length: 76, dtype: int64

In [163]:
df.drop('Cabin' , axis=1,inplace=True)

In [165]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Embarked'],
      dtype='object')

In [167]:
df.drop(['PassengerId','Ticket','Name'],axis=1,inplace=True)

In [169]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,34.5,0,0,7.8292,Q
1,1,3,female,47.0,1,0,7.0000,S
2,0,2,male,62.0,0,0,9.6875,Q
3,0,3,male,27.0,0,0,8.6625,S
4,1,3,female,22.0,1,1,12.2875,S


In [75]:
x = df.iloc[:,1:]

In [77]:
y=df.iloc[:,0]

In [79]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=.20,random_state=42)

In [105]:
# for filling missing vule we use simpleimputer

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OneHotEncoder

In [91]:
trf1 = ColumnTransformer([
    ('impute',SimpleImputer(),[2]), # (name of tranform , object of tranformation,column index) 
    ('impute1',SimpleImputer(strategy = 'most_frequent'),[6])
],remainder = 'passthrough')
    

In [95]:
trf1.fit_transform(x_train).shape

(334, 7)

In [97]:
trf1.named_transformers_

{'impute': SimpleImputer(),
 'impute1': SimpleImputer(strategy='most_frequent'),
 'remainder': FunctionTransformer(accept_sparse=True, check_inverse=False,
                     feature_names_out='one-to-one')}

In [103]:
trf1.named_transformers_['impute1'].statistics_

array(['S'], dtype=object)

In [115]:
trf2 = ColumnTransformer([
    ('ohe',OneHotEncoder(sparse_output = False ,handle_unknown = 'ignore'),[1,6])
])
 

In [117]:
from xgboost import  XGBClassifier

In [119]:
clf = XGBClassifier()

In [121]:
from sklearn.pipeline import Pipeline

In [123]:
pipe = Pipeline([
    ('step1',trf1),
    ('step2',trf2),
    ('step3',clf)       
])

In [125]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute', SimpleImputer(),
                                                  [2]),
                                                 ('impute1',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('step2',
                 ColumnTransformer(transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('step3',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbac...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=6, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=0, num_parallel_tree=1,
                               predictor='auto', random_state=0, reg_alpha=0,
                               reg_lambda=1, ...))])

In [129]:
y_pred=pipe.predict(x_test)

In [131]:
from sklearn.metrics import accuracy_score

In [133]:
accuracy_score(y_test , y_pred)

0.5595238095238095

In [135]:
import pickle

In [137]:
pickle.dump(pipe , open('pipe.pkl' ,'wb'))

In [143]:
a=pickle.load(open('pipe.pkl' ,'rb'))

In [183]:
print(a.predict(np.array([[3,	'male'	,34.5,	0	,0,	7.8292,	'Q']])))

[1]


D:\anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
D:\anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
